# Претурање вода во садови
Имате на располагање 3 садови. Нивниот капацитет е 5, 8 и 10 соодветно. На почетокот сите садови се празни. Задачата е да претурите 1 литар во садот од 5 литри. Можете да претурате вода меѓу садовите, но не знаете точно колку претурате бидејќи садовите не се обележани. Единствено може да го испразните садот од кој претурате или да го наполните до врв садот во кој претурате. Постои и чешма од која може да надополните некој сад до врвот. Секогаш може да ја истурите водата од некој сад. Скратено: имате на располагање 3 садови. Нивниот капацитет е (x, y, z) соодветно. Почетната состојба е (x_0, y_0, z_0). Задачата е да стигнете до посакуваната состојба.

In [1]:
cap=(5,8,10)

In [2]:
from collections import deque
from itertools import combinations

In [3]:
def transfer(current, sink, source):
    current=list(current)
    available = cap[sink]-current[sink]
    newsource = max(current[source] - available,0)
    newsink = min(current[source] + current[sink],cap[sink])
    new_current = current
    new_current[source] = newsource
    new_current[sink] = newsink
    return tuple(new_current)

In [4]:
transfer((4, 5, 10), 0, 1)

(5, 4, 10)

In [5]:
def expand_state(state):
    new_state=[]
    for index, element in enumerate(state):
        lista=list(state)
        if element>0:
            lista[index]=0
            new_state.append(tuple(lista))
    for index, element in enumerate(state):
        lista=list(state)
        if element<cap[index]:
            lista[index]=cap[index]
            new_state.append(tuple(lista))
    for i1, element1 in enumerate(state):
        for i2, element2 in enumerate(state):
            if i1!=i2:
                if transfer(state,i1,i2) not in new_state:
                    new_state.append(transfer(state,i1,i2))
    return tuple(new_state)

In [6]:
expand_state((2, 0, 0))

((0, 0, 0), (5, 0, 0), (2, 8, 0), (2, 0, 10), (2, 0, 0), (0, 2, 0), (0, 0, 2))

In [35]:
start=(5, 8, 10)
expand_state(start)

((0, 8, 10), (5, 0, 10), (5, 8, 0), (5, 8, 10))

In [36]:
def search(initial_state, goal_state, alg):
    visited = {initial_state}
    queue = deque([[initial_state]])
    while queue:
        vertex_list = queue.popleft()
        state_to_expand = vertex_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state==goal_state:
                    return vertex_list + [next_state]
                visited.add(next_state)
                if alg == 'dfs':
                    queue.appendleft(vertex_list + [next_state])
                elif alg == 'bfs':
                    queue.append(vertex_list + [next_state])

In [39]:
initial_state = (0, 0, 0)
goal_state = (1, 0, 0)
path = search(initial_state, goal_state,'bfs')
path

[(0, 0, 0),
 (0, 8, 0),
 (5, 3, 0),
 (0, 3, 0),
 (3, 0, 0),
 (3, 8, 0),
 (3, 0, 8),
 (1, 0, 10),
 (1, 0, 0)]

Дополнително, секој нов наточен литар се казнува со 1 поен (еквивалентно на 1 потег), а секој истурен литар се казнува со 3 поени (еквивалетно на 3 потези).

In [45]:
def expand_state(state):
    new_state=[]
    for index, element in enumerate(state):
        lista=list(state)
        if element>0:
            lista[index]=0
            new_state.append((3*state[index],tuple(lista)))
    for index, element in enumerate(state):
        lista=list(state)
        if element<cap[index]:
            lista[index]=cap[index]
            new_state.append((cap[index]-state[index],tuple(lista)))
    for i1, element1 in enumerate(state):
        for i2, element2 in enumerate(state):
            if i1!=i2:
                if transfer(state,i1,i2) not in new_state:
                    new_state.append((0,transfer(state,i1,i2)))
    return tuple(new_state)

In [46]:
expand_state((2, 0, 0))

((6, (0, 0, 0)),
 (3, (5, 0, 0)),
 (8, (2, 8, 0)),
 (10, (2, 0, 10)),
 (0, (2, 0, 0)),
 (0, (2, 0, 0)),
 (0, (0, 2, 0)),
 (0, (2, 0, 0)),
 (0, (0, 0, 2)),
 (0, (2, 0, 0)))

In [54]:
import heapq
def uniform_cost_search(starting_vertex, goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue) 
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list
        if vertex_to_expand not in expanded:
            for new_weight, neighbour in expand_state(vertex_to_expand):
                if neighbour not in expanded: 
                    heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [55]:
uniform_cost_search((0, 0, 0), (1, 0, 0))

(41,
 [(0, 0, 0),
  (0, 0, 10),
  (0, 8, 2),
  (2, 8, 0),
  (2, 0, 8),
  (0, 2, 8),
  (5, 2, 3),
  (0, 7, 3),
  (0, 8, 3),
  (0, 1, 10),
  (0, 1, 0),
  (0, 0, 1),
  (1, 0, 0)])

# Поставување кралици на шаховска табла

Распоредете N кралици на NxN шаховска табла така што кралиците нема да се напаѓаат.

In [1]:
from collections import deque
import os

In [13]:
N=4 #bilo koj broj sakame da e tablata
g=(None,None,None,None,
   'x', None, None, None,
   None, 'x', None, 'x',
   None, 'x', 'x', None)

In [16]:
def valid(state,old_queen,new_queen):
    index1,index2=old_queen
    i,j=new_queen
    states=list(state)
    if states[index1][index2]!=None:
        if states[i+1][j]!=None:
            return False
        if states[i][j+1]!=None:
            return False
        if states[i-1][j]!=None:
            return False
        if states[i][j-1]!=None:
            return False
        if states[i+1][j-1]!=None:
            return False
        if states[i+1][j+1]!=None:
            return False
        if states[i-1][j+1]!=None:
            return False
        if states[i-1][j-1]!=None:
            return False
        return True

In [19]:
valid(g,(1,0),(1,3))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def expand_state(state):
    states=[]
    available=state.count(None)
    if available==0:
        return []
    for i in range(N):
        for j in range(N):
            if valid(state,i,j):
                states[i][j]=
            #kancer mi e zadachava

# Sudoku

In [26]:
from collections import deque
import os
from pprint import pprint
import math
import copy

In [27]:
def valid(state, number, row, col):
    if number in state[row]:
        return False

    for row in state:
        if row[col]==number:
            return False
        
    square_list = [
        [[], [], []],
        [[], [], []],
        [[], [], []]
    ]    
    square_row = math.floor(row / 3)
    square_col = math.floor(col / 3)    
    for i in range(9):
        for j in range(9):
            square_i = math.floor(i / 3)
            square_j = math.floor(j / 3)
            square_list[square_i][square_j].append(state[i][j]) 
    if number in square_list[square_row][square_col]:
        return False
    
    return True

In [13]:
state = (
    (None, None, None, None, 5, 6, 7, 8, None),
    (1, None, None, None, None, None, 3, None, None),
    (2, 7, None, 3, None, None, None, None, None),
    (3, None, None, None, None, None, 9, None, None),
    (4, None, None, None, None, None, None, None, 2),
    (None, None, 9, None, None, None, None, None, 3),
    (None, None, None, None, None, 7, None, 5, 4),
    (None, None, 2, None, None, None, None, None, 8),
    (None, 4, 5, 6, 3, None, None, None, None)
)

In [10]:
def expand_state(state):
    states=list(state)
    for i in range(9):
        for j in range(9):
            if state[i][j]!= None:
                for number in range(10):
                    if valid(state, number, i, j):
                        states[i][j]=number
    new_state.append(tuple(states))
    return new_state

In [11]:
def depth_first_search_find_path(starting_state):
    visited = {starting_state}
    queue = deque([[starting_state]])
    while queue:
        state_list = queue.popleft()
        state_to_expand = state_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                zeros=0
                for row in next_state:
                    zeros += row.count(None)
                if zeros == 0:
                    return next_state
                visited.add(next_state)
                queue.appendleft(state_list + [next_state])
    return []

In [3]:
from time import sleep
from collections import deque
from copy import deepcopy
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

In [4]:
class Game:
    def __init__(self, player_1, player_2):
        self.player_1 = {**player_1, 'symbol': 'x', 'role': 'MAX'}
        self.player_2 = {**player_2, 'symbol': 'o', 'role': 'MIN'}
        self.next_to_play = self.player_1
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]
        self.symbols_fig = {'x': 'x', 'o': 'circle', '·': 'circle-open'}
        self.scores = {'x': 1, 'o': -1, 'draw': 0}
        self.create_ui()
        hbox = widgets.HBox([self.bt_reset, self.dashboard])
        display(widgets.VBox([hbox, self.fig]))
        self.reset()

    def reset(self, *args):
        self.next_to_play = self.player_1
        self.update_score('На ред е', self.next_to_play)
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()
        self.evaluated = {}
        self.winner = 'keep_playing'
        self.initiate_turn()

    def create_ui(self):
        self.dashboard = widgets.HTML(description='Статус:', value='')
        self.bt_reset = widgets.Button(description='Ресетирај')
        self.bt_reset.on_click(self.reset)
        self.fig = self.create_fig()

    def create_fig(self):
        N = 3
        fig = go.FigureWidget()
        x = [x for y in range(N) for x in range(N)]
        y = [y for y in range(N) for x in range(N)]
        symbols = [self.symbols_fig[v] for row in self.state for v in row]
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
        fig.data[0].on_click(self.human_move)
        fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(
            range=[-0.5, N - 0.5], dtick=1, title='y', autorange='reversed')
        fig.update_layout(width=600, height=600, showlegend=False)
        return fig

    def convert_state_to_symbols(self):
        return [self.symbols_fig[v] for row in self.state for v in row]

    def initiate_turn(self):
        if 'human' not in [self.player_1['type'], self.player_2['type']]:
            while self.winner == 'keep_playing':
                self.ai_move()
        elif self.next_to_play['type'] == 'AI':
            self.ai_move()

    def ai_move(self):
        self.dashboard.value += ' -- пресметува'
        state = tuple([tuple(row) for row in self.state])
        result, move = self.minimax(state, self.next_to_play['role'])
        sleep(1)
        self.player_took_turn(*move)

    def human_move(self, trace, points, selector):
        x, y = points.xs[0], points.ys[0]
        keep_playing = self.winner == 'keep_playing'
        human_on_turn = self.next_to_play['type'] == 'human'
        empty_place = self.state[y][x] == '·'
        if keep_playing and human_on_turn and empty_place:
            self.player_took_turn(x, y)
            if self.next_to_play['type'] == 'AI':
                self.ai_move()

    def player_took_turn(self, x, y):
        self.state[y][x] = self.next_to_play['symbol']
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()
        self.winner = self.check_victory(self.state)
        if self.winner != 'keep_playing':
            if self.winner == 'draw':
                self.dashboard.value = 'Нерешено.'
            else:
                self.update_score('Победник е', self.next_to_play)
            return
        if self.next_to_play == self.player_2:
            self.next_to_play = self.player_1
        else:
            self.next_to_play = self.player_2
        self.update_score('На ред е', self.next_to_play)

    def update_score(self, message, player):
        player_data = ' - '.join(list(player.values())[:-1])
        self.dashboard.value = f'{message} <b> {player_data} </b>.'

    def minimax(self, node, player, alpha=-2, beta=2, depth=0):
        if node not in self.evaluated:
            self.evaluated[node] = self.check_victory(node)
        if self.evaluated[node] != 'keep_playing':
            return self.scores[self.evaluated[node]], None
        best_value = 2 if player == 'MIN' else -2
        best_move = None
        for child, move in self.expand_state(node, player):
            other_player = 'MIN' if player == 'MAX' else 'MAX'
            result, _ = self.minimax(child, other_player, alpha, beta, depth+1)
            if player == 'MIN':
                if result <= alpha:
                    return result, best_move
                if result < beta:
                    beta = result
                if result < best_value:
                    best_value = result
                    best_move = move
            elif player == 'MAX':
                if result >= beta:
                    return result, best_move
                if result > alpha:
                    alpha = result
                if result > best_value:
                    best_value = result
                    best_move = move
        return best_value, best_move

# Plochki

In [1]:
from collections import deque
from pprint import pprint

In [2]:
state = ((0, 5, 3),
         (7, 8, 4),
         (2, 6, 1))

goal_state = ((0, 1, 2),
              (3, 4, 5),
              (6, 7, 8))


In [6]:
def move(state,currentp,zerop):
    crow,ccol=currentp
    zrow,zcol=zerop
    states=[]
    statet=[]
    for row in state:
        states.append(list(row))
    states[zrow][zcol]=states[crow][ccol]
    states[crow][ccol]=0
    for row in states:
        statet.append(tuple(row))
    return tuple(statet)

In [7]:
move(state,(2,1),(0,0))

((6, 5, 3), (7, 8, 4), (2, 0, 1))

In [ ]:
def expand_state(state):
    states=[]
    for indeks,row in enumerate(state):
        if 0 in row
        zerop=(indeks,row.index(0))
        break
    zrow,zcol=zerop
    tile_indexes_to_move = [(zero_row+1, zero_col),
                            (zero_row-1, zero_col),
                            (zero_row, zero_col+1),
                            (zero_row, zero_col-1)]    
    for indexr, indexc in tile_indexes_to_move:
        if (indexr<0 or indexr >=N or indexc<0 or indexc >=N):
            continue
        elif states.append((1,move(state,(indexr,indexc),zerop)
    return states
    

In [ ]:
def search(initial_state, alg):
    visited = {initial_state}
    queue = deque([[initial_state]])
    c = 0
    while queue:
        c += 1
        vertex_list = queue.popleft()
        state_to_expand = vertex_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if check_end(next_state):
                    return vertex_list + [next_state], c
                visited.add(next_state)
                if alg == 'dfs':
                    queue.appendleft(vertex_list + [next_state])
                elif alg == 'bfs':
                    queue.append(vertex_list + [next_state])

употреби сега А* алгоритам и како подобро работи и колку помалку јазли ќе разграни

In [ ]:
gp= {
    0: (0, 0), 1: (0, 1), 2: (0, 2),
    3: (1, 0), 4: (1, 1), 5: (1, 2),
    6: (2, 0), 7: (2, 1), 8: (2, 2)
}

bgp= {
    0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3),
    4: (1, 0), 5: (1, 1), 6: (1, 2), 7: (1, 3),
    8: (2, 0), 9: (2, 1), 10: (2, 2), 11: (2, 3),
    12: (3, 0), 13: (3, 1), 14: (3, 2), 15: (3, 3)
}

In [13]:
import heapq

In [15]:
def heuristics(state):
    value = 0
    for index_row, row in enumerate(state):
        for index_col, col in enumerate(row):
            value += manhattan_distance((index_row, index_col), col)
    return value

In [16]:
def manhattan_distance(state_1, n):
    return abs(state_1[0] - bgp[n][0]) + abs(state_1[1] - bgp[n][1])

In [ ]:
def a_star_search(graph, starting_vertex, goal_vertex, heuristic_function, alpha=1):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded:
                    heuristic = heuristic_function(neighbour, goal_vertex)
                    path_weight = current_path_weight + new_weight
                    a_star_weight = path_weight + alpha * heuristic
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

# Preminuvanje reka
Сточар, момче, девојче, куче, две верверички и две зајчиња треба да ја преминат реката.
    * Ако сточарот не е присутен кучето ќе ги искаса сите присутни.
    * Ако момчето не е присутно девојчето ќе ги закача зајчињата.
    * Ако девојчето не е присутно момчето ќе ги закача верверичките.
    * Само луѓето можат да веслаат
    * Чамецот може да издржи највеќе двајца.

In [1]:
# state kje bide so indeksi:0=stochar 1=momche 2=devojche 3=kuche 4=prva ververica 5=vtora ververica 6=prv zajak 7=vtor zajak,8=chamecot e na drta strana od rekata
#toa znaci lista so 9 promenlivi
stocar,momce,devojce,kuce,prvaver,vtoraver,prvz,vtorz,camec=range(9)

In [2]:
def one_person_goes(state,indeks):
    states=list(state)
    states[8]=states[indeks]=1
    return tuple(states)

In [3]:
def two_person_goes(state,indeks1,indeks2):
    states=list(state)
    states[8]=states[indeks1]=states[indeks2]=1
    return tuple(states)

In [4]:
def one_person_comes(state,indeks):
    states=list(state)
    states[8]=states[indeks]=0
    return tuple(states)

In [5]:
def two_person_comes(state,indeks1,indeks2):
    states=list(state)
    states[8]=states[indeks1]=states[indeks2]=0
    return tuple(states)

In [6]:
def expand_state(state):
    states=[]
    if state[8] == 0:
        if state[0] == 0 and state[3] == 0:  
            states.append(two_person_goes(state,0,3))
        for indeks in range(9):
            if state[1] == 0 and state[3] != 0 and state[4] != 0 and state[5] != 0:
                states.append(two_person_goes(state,1,indeks))
            if state[2] == 0 and state[3] != 0 and state[7]!=0 and state[8]!=0:
                states.append(two_person_goes(state,2,indeks))
            if state[0]==0:
                states.append(two_person_goes(state,0,indeks))
        for indeks in [stocar,momce,devojce]:
            if state[indeks]==0:
                states.append(one_person_goes(state,indeks))
    if state[8] == 1:
        if state[0]==1 and state[3]==1:
            states.append(two_person_comes(state,0,3))
        for indeks in range(9):
            if state[1]==1 and state[3]!=1 and state[4]!=1 and state[5]!=1:
                states.append(two_person_comes(state,1,indeks))
            if state[2]==1 and state[3]!=1 and state[7]!=1 and state[8]!=1:
                states.append(two_person_comes(state,2,indeks))
            if state[0]==1:
                states.append(two_person_comes(state,0,indeks))
        for indeks in [stocar,momce,devojce]:
                if state[indeks]==1:
                    states.append(one_person_comes(state,indeks))
    return states

In [7]:
from collections import deque
def breadth_first_search_find_path(starting_vertex, goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    visited = {starting_vertex} 
    queue = deque([[starting_vertex]]) 
    while queue:
        vertex_list = queue.popleft() 
        vertex_to_expand = vertex_list[-1]
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                if neighbour == goal_vertex:
                    return vertex_list + [neighbour]
                visited.add(neighbour)
                queue.append(vertex_list + [neighbour])

In [8]:
breadth_first_search_find_path((0,0,0,0,0,0,0,0,0), (1,1,1,1,1,1,1,1,1))

[(0, 0, 0, 0, 0, 0, 0, 0, 0),
 (1, 0, 0, 1, 0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 0, 0, 0, 0),
 (1, 1, 0, 1, 0, 0, 0, 0, 1),
 (0, 1, 0, 1, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 0, 0, 0, 0, 1),
 (0, 1, 1, 1, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 0, 0, 0, 1),
 (0, 1, 1, 1, 1, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 0, 0, 1),
 (0, 1, 1, 1, 1, 1, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 0, 1),
 (0, 1, 1, 1, 1, 1, 1, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 1, 1)]

# Најкратка патека за коњот во шахот
Користејќи ја шаховската фигура коњ, најдете ја најкратката патека од едно до друго поле на шаховската табла.
"""
    Коњот се движи во Г форма, односно: или оди 2 полиња вертикално
    (нагоре (+) и надолу(-)), а 1 поле хоризонтално (налево (-) и надесно(+)),
    или обратно: 1 поле вертикално, а 2 полиња хоризонтално.
    Елементите од листата `possible_moves` се торки чиј прв елемент ќе се
    додаде на моменталниот ред каде се наоѓа коњот, а вториот елемент од
    торката ќе се додаде на моменталната колона каде се наоѓа коњот.
    Вака ќе симулираме придвижување на коњот по шаховската табла.
    """

In [45]:
N=8

In [46]:
from collections import deque
def expand_state(state):
    i,j=state
    states=[]
    possible_moves = [(i-1, j-2), (i-1, j+2), (i+1, j-2), (i+1, j+2),
                      (i-2, j-1), (i-2, j+1), (i+2, j-1), (i+2, j+1)]
    for x,y in possible_moves:
        if x>=0 and x<N and y>=0 and y<N:
            states.append((x,y))
    return states

In [47]:
def breadth_first_search_find_path(starting_vertex, goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    visited = {starting_vertex} 
    queue = deque([[starting_vertex]]) 
    while queue:
        vertex_list = queue.popleft() 
        vertex_to_expand = vertex_list[-1]
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                if neighbour == goal_vertex:
                    return vertex_list + [neighbour]
                visited.add(neighbour)
                queue.append(vertex_list + [neighbour])

In [48]:
breadth_first_search_find_path((0, 0), (3, 7))

[(0, 0), (1, 2), (0, 4), (1, 6), (3, 7)]

# Distances in Mk
Вчитајте ги податоците од **distances.txt**, создадете граф и употребете алгоритам за да ја најде најоптималната патека од еден до друг град.


In [3]:
class WeightedGraph:
    def __init__(self):
        self.graph_dict = {}
    
    def add_vertex(self, vertex):
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = {}
    
    def vertices(self):
        return list(self.graph_dict.keys())
    
    def add_edge(self, edge, add_reversed=True):
        vertex1, vertex2, weight = edge
        self.graph_dict[vertex1][vertex2] = weight
        if add_reversed:
            self.graph_dict[vertex2][vertex1] = weight
    
    def edges(self):
        edges = []
        for vertex in self.graph_dict:
            for neighbour, weight in self.graph_dict[vertex].items():
                edges.append((vertex, neighbour, weight))
        return edges
    
    def neighbours(self, vertex):
        return list(self.graph_dict[vertex].items())

In [4]:
info = open('data/distances.txt').read().strip()
g= WeightedGraph()

In [5]:
for row in info.split('\n'):
    vertex1,vertex2,weight=row.split(' ')
    g.add_vertex(vertex1)
    g.add_vertex(vertex2)
    g.add_edge(row.split(' '))

In [6]:
g.edges()

[('KU', 'SK', '39'),
 ('KU', 'KP', '61'),
 ('KU', 'KR', '51'),
 ('SK', 'KU', '39'),
 ('SK', 'TE', '44'),
 ('SK', 'VE', '55'),
 ('TE', 'SK', '44'),
 ('TE', 'GV', '24'),
 ('GV', 'TE', '24'),
 ('GV', 'KI', '46'),
 ('GV', 'DB', '68'),
 ('KI', 'GV', '46'),
 ('KI', 'OH', '61'),
 ('KI', 'KV', '62'),
 ('DB', 'GV', '68'),
 ('DB', 'SU', '52'),
 ('SU', 'DB', '52'),
 ('SU', 'OH', '15'),
 ('OH', 'KI', '61'),
 ('OH', 'SU', '15'),
 ('OH', 'RS', '35'),
 ('KV', 'KI', '62'),
 ('KV', 'PP', '32'),
 ('RS', 'OH', '35'),
 ('RS', 'BT', '30'),
 ('BT', 'RS', '30'),
 ('BT', 'PP', '47'),
 ('PP', 'KV', '32'),
 ('PP', 'BT', '47'),
 ('PP', 'NG', '61'),
 ('NG', 'PP', '61'),
 ('NG', 'VE', '41'),
 ('NG', 'GE', '64'),
 ('VE', 'SK', '55'),
 ('VE', 'NG', '41'),
 ('VE', 'ST', '51'),
 ('ST', 'VE', '51'),
 ('ST', 'KR', '52'),
 ('ST', 'KO', '32'),
 ('ST', 'RA', '29'),
 ('ST', 'BR', '54'),
 ('ST', 'VV', '22'),
 ('GE', 'NG', '64'),
 ('GE', 'DR', '34'),
 ('KP', 'KU', '61'),
 ('KP', 'KR', '32'),
 ('KR', 'KU', '51'),
 ('KR', 'KP',

In [7]:
g.neighbours('SK')

[('KU', '39'), ('TE', '44'), ('VE', '55')]

In [8]:
import heapq
def uniform_cost_search(graph,starting_vertex, goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue) 
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in g.neighbours(vertex_to_expand):
                if neighbour not in expanded: 
                    heapq.heappush(queue, (weight + int(new_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [9]:
uniform_cost_search(g, 'GE', 'TE')

(204, ['GE', 'NG', 'VE', 'SK', 'TE'])

# Chrossing a bridge
Adam, Bob, Clair and Dave are out walking: They come to rickety old wooden bridge. The bridge is weak and only able to carry the weight of two of them at a time. Because they are in a rush and the light is fading they must cross in the minimum time possible and must carry a torch (flashlight,) on each crossing.

They only have one torch and it can't be thrown. Because of their different fitness levels and some minor injuries they can all cross at different speeds. Adam can cross in 1 minute, Bob in 2 minutes, Clair in 5 minutes and Dave in 10 minutes.

Adam, the brains of the group thinks for a moment and declares that the crossing can be completed in 17 minutes. There is no trick. How is this done?

In [80]:
#adam,bob clar i dave kje se indeks 0-3 a torchot kje e 4
#mozhe da projdat max 2ca, ama torchot mora da e vo raka pri sekoe pominuvanje
minutes=(1,2,5,10)

In [94]:
def cross(state,index,number):
    new_state=list(state)
    if state[4]==0:
        if number==1:
            new_state[index]=new_state[4]=1
            return minutes[index],tuple(new_state)
        if number==2:
            new_state[index[0]]=new_state[index[1]]=new_state[4]=1
            return max(minutes[index[0]],minutes[index[1]]),tuple(new_state)

In [95]:
def back(state,index,number):
    new_state=list(state)
    if state[4]==1:
        if number==1:
            new_state[index]=new_state[4]=0
            return minutes[index],tuple(new_state)
        if number==2:
            new_state[index[0]]=new_state[index[1]]=new_state[4]=0
            return max(minutes[index[0]],minutes[index[1]]),tuple(new_state)

In [106]:
def expand_state(state):
    states=[]
    if state[4] == 0:
        for index1 in range(4):
            if state[index1]==0:
                states.append(cross(state,index1,1))
        for index1 in range(4):
            for index2 in range(4):
                if index1!=index2 and state[index1]==state[index2]==0: 
                    states.append(cross(state,[index1,index2],2))          
    if state[4] == 1:
        for index1 in range(4):
            if state[index1]==1:
                states.append(back(state,index1,1))
        for index1 in range(4):
            for index2 in range(4):
                if index1!=index2 and state[index1]==state[index2]==1: 
                    states.append(back(state,[index1,index2],2))                          
    return tuple(states)   

In [107]:
expand_state((0, 0, 0, 0, 0))

((1, (1, 0, 0, 0, 1)),
 (2, (0, 1, 0, 0, 1)),
 (5, (0, 0, 1, 0, 1)),
 (10, (0, 0, 0, 1, 1)),
 (2, (1, 1, 0, 0, 1)),
 (5, (1, 0, 1, 0, 1)),
 (10, (1, 0, 0, 1, 1)),
 (2, (1, 1, 0, 0, 1)),
 (5, (0, 1, 1, 0, 1)),
 (10, (0, 1, 0, 1, 1)),
 (5, (1, 0, 1, 0, 1)),
 (5, (0, 1, 1, 0, 1)),
 (10, (0, 0, 1, 1, 1)),
 (10, (1, 0, 0, 1, 1)),
 (10, (0, 1, 0, 1, 1)),
 (10, (0, 0, 1, 1, 1)))

In [108]:
import heapq

In [109]:
def uniform_cost_search(starting_vertex,goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue) 
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list
        if vertex_to_expand not in expanded:
            for new_weight, neighbour in expand_state(vertex_to_expand):
                if neighbour not in expanded: 
                    heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [110]:
uniform_cost_search((0, 0, 0, 0, 0), (1, 1, 1, 1, 1))

(17,
 [(0, 0, 0, 0, 0),
  (1, 1, 0, 0, 1),
  (0, 1, 0, 0, 0),
  (0, 1, 1, 1, 1),
  (0, 0, 1, 1, 0),
  (1, 1, 1, 1, 1)])

# Pivara
Разгледуваме фабрика за производство на пиво. За пакување на пивото во шишиња се користат рециклирани шишиња кои во фабриката пристигаат во гајби. Следат постапките:

1. Сите шишиња се ставаат врз платформата.
    - Платформата собира 15х15 шишиња.
    - За поедноставни пресметки за време на испитот, ќе сметаме дека платформата собира 7х7 шишиња.
2. Софтвер за препознавање ги скенира сите шишиња од платформата, и во себе забележува по еден код за секое шише:
    - (0) - Празно место, нема шише.
    - (1) - Скршено шише.
    - (2) - Шише во добра состојба, од нашата пивара.
    - (3) - Шише од друга пивара.
3. Роботот ги избира соодветните шишиња (со код 2) и ги префрла на подвижната лента за понатаму.
    - Роботот наеднаш може да земе 10 шишиња, бидејќи има 10 дланки (5 х 2).
    - За поедноставни пресметки за време на испитот, ќе сметаме дека роботот има 6 дланки (3 х 2), па роботот може наеднаш да земе најмногу 6 шишиња од платформата.
    - Роботот може да ротира за 90 степени.

Ваша задача е да напишете код со алгоритмите од вештачка интелигенција за чекорот 3. Целта е роботот во минимален број придвижувања да ги издвои соодветните шишиња (со код 2).
### Важно при решавањето
- Дефинираме состојбата да биде матрица 7х7 (торка од торки), како што е дефинирана променливата `g` подолу.
- За роботот сите придвижувања имаат еднаква тежина во пребарување.
    - Едно придвижување се дефинира како:
        - Чекор 1 - поместување
        - Чекор 2 - кревање шишиња
        - Чекор 3 - повторно поместување
        - Чекор 4 - ставање на шишињата во гајбата


**Нотација**
- ППШ = Пребарување Прво по Широчина
- ППД = Пребарување Прво по Длабочина


In [1]:
g  = (
    (0, 3, 2, 0, 2, 2, 2),
    (2, 1, 2, 2, 2, 3, 2),
    (2, 2, 0, 2, 2, 2, 2),
    (0, 2, 2, 2, 2, 2, 0),
    (2, 2, 3, 2, 2, 2, 2),
    (2, 2, 1, 2, 2, 2, 2),
    (0, 3, 0, 2, 2, 2, 2)
)

се избират шишињата со код 2
0 = празно место
3х2 област на земање шишиња или 2х3
роботот да ги издвои шишињата со код 2 во најмалку чекори
1 чекор е да се претвори матрицата од 2 во 0

In [2]:
def goal_state(state):
    
    state_list = []
    for row in state:
        state_list.append(list(row))
        
    for row_index, row in enumerate(state_list):
        for col_index, element in enumerate(row):
            if element == 2:
                state_list[row_index][col_index] = 0
                
    state_tuple = []
    for row in state_list:
        state_tuple.append(tuple(row))
        
    return tuple(state_tuple)

In [3]:
def valid(state):
    if state[0]>=0 and state[0]<N and state[1]>=0 and state[1]<N:
        return True
    else: return False

In [4]:
def bottle(state,grid,position):
    gridrow,gridcol=grid
    posrow,poscol=position
    states=[]
    for row in state:
        states.append(list(row))
    for i in range(gridrow):
        for j in range(gridcol):
            if valid(states[posrow+i][poscol+j]):
                if states[posrow+1][poscol+1]==2:
                    states[posrow+1][poscol+1]==0
    state=[]
    for row in states:
        state.append(tuple(row))        
    return state

In [5]:
def expand_state(state,grid):
    states=[]
    grid=(2,3)
    for row in range(-1,7):
        for col in range(-2,7):
            if valid(state[row][col]):
                if bottle(state,grid,(row,col)) not in states:
                    states.append(bottle(state,grid,(i,j)))
    grid=(2,3)
    for row in range(-2,7):
        for col in range(-1,7):
            if valid(state[row][col]):
                if bottle(state,grid,(row,col)) not in states:
                    states.append(bottle(state,grid,(i,j)))
    return states

In [6]:
from queue import deque
def depth_first_search_find_path(starting_vertex,grid,goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        for neighbour in expand_state(vertex_to_expand,grid): 
            if neighbour not in visited:
                if neighbour == goal_vertex:
                    return vertex_list + [neighbour]
                visited.add(neighbour)
                queue.appendleft(vertex_list + [neighbour])

In [7]:
goal_vertex=goal_state(g)
depth_first_search_find_path(g,(3,2),goal_vertex)

TypeError: 'int' object is not subscriptable

b)sekoj poteg e tezhina 1

In [12]:
def expand_state(state,grid):
    states=[]
    grid=(2,3)
    for row in range(-1,7):
        for col in range(-2,7):
            if valid(state[row][col]):
                if bottle(state,grid,(row,col)) not in states:
                    states.append((1,bottle(state,grid,(i,j))))
    grid=(2,3)
    for row in range(-2,7):
        for col in range(-1,7):
            if valid(state[row][col]):
                if bottle(state,grid,(row,col)) not in states:
                    states.append((1,bottle(state,grid,(i,j))))
    return states

In [13]:
import heapq
def uniform_cost_search(starting_vertex, grid, goal_vertex):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue) 
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list
        if vertex_to_expand not in expanded:
            for new_weight, neighbour in expand_state(vertex_to_expand):
                if neighbour not in expanded: 
                    heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

v) A* i napravi evristika shto ima smisla

In [14]:
import math
def heuristic(state):
    twos=0
    for row in state:
        twos+=row.count(2)
    return math.ceil(twos/6)

In [15]:
def a_star_search(starting_vertex, goal_vertex,alpha=1):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in expand_state(vertex_to_expand):
                if neighbour not in expanded:
                    heuristich = heuristic(neighbour)
                    path_weight = current_path_weight + new_weight
                    a_star_weight = path_weight + alpha * heuristich
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

# Tezhinski grafik i A*

![](images/A_star.png)

In [116]:
class WeightedGraph:
    def __init__(self):
        self.graph_dict = {}
    
    def add_vertex(self, vertex):
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = {}
    
    def vertices(self):
        return list(self.graph_dict.keys())
    
    def add_edge(self, edge, add_reversed=True):
        vertex1, vertex2, weight = edge
        self.graph_dict[vertex1][vertex2] = weight
        if add_reversed:
            self.graph_dict[vertex2][vertex1] = weight
    
    def edges(self):
        edges = []
        for vertex in self.graph_dict:
            for neighbour, weight in self.graph_dict[vertex].items():
                edges.append((vertex, neighbour, weight))
        return edges
    
    def neighbours(self, vertex):
        return list(self.graph_dict[vertex].items())

In [117]:
g=WeightedGraph()
g.add_vertex('A')
g.add_vertex('B')
g.add_vertex('C')
g.add_vertex('D')
g.add_vertex('E')
g.add_vertex('F')
g.add_vertex('G')
g.add_vertex('H')
g.add_vertex('S')

g.add_edge(('A','C',2))
g.add_edge(('A', 'S', 10))
g.add_edge(('B', 'D', 8))
g.add_edge(('B', 'G', 16))
g.add_edge(('B', 'S', 8))
g.add_edge(('C', 'E', 3))
g.add_edge(('C', 'G', 9))
g.add_edge(('D', 'H', 1))
g.add_edge(('D', 'G', 3))
g.add_edge(('E', 'G', 2))
g.add_edge(('F', 'H', 1))

In [118]:
import heapq
def a_star_search(graph, starting_vertex, goal_vertex, heuristic_function, alpha=1):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded:
                    heuristic = heuristic_function(neighbour, goal_vertex)
                    path_weight = current_path_weight + new_weight
                    a_star_weight = path_weight + alpha * heuristic
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [119]:
d_heuristic = {'S': 12, 'A': 5, 'B': 5, 'C': 5, 'D': 2, 'E': 2, 'F': 1, 'G': 1, 'H': 0} 
def heuristic(vertex, goal_vertex):
    return d_heuristic[vertex]

In [121]:
a_star_search(g, starting_vertex='A', goal_vertex='G', heuristic_function=heuristic)

(7, ['A', 'C', 'E', 'G'])

# Locations
Вчитајте ги податоците од **locations.txt**, складирајте ги соодветно и напишете функција која го пресметува воздушното растојание помеѓу два града. Потоа напишете А* алгоритам кој како еристика го користи воздушното растојание до посакуваниот град. Задачата е иста како од претходната лабораториска вежба: да се најде најоптималната патека од еден до друг град. 

In [1]:
class WeightedGraph:
    def __init__(self):
        self.graph_dict = {}
    
    def add_vertex(self, vertex):
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = {}
    
    def vertices(self):
        return list(self.graph_dict.keys())
    
    def add_edge(self, edge, add_reversed=True):
        vertex1, vertex2, weight = edge
        self.graph_dict[vertex1][vertex2] = weight
        if add_reversed:
            self.graph_dict[vertex2][vertex1] = weight
    
    def edges(self):
        edges = []
        for vertex in self.graph_dict:
            for neighbour, weight in self.graph_dict[vertex].items():
                edges.append((vertex, neighbour, weight))
        return edges
    
    def neighbours(self, vertex):
        return list(self.graph_dict[vertex].items())

In [2]:
info = open('data/distances.txt').read().strip()
g= WeightedGraph()

In [3]:
for row in info.split('\n'):
    vertex1,vertex2,weight=row.split(' ')
    g.add_vertex(vertex1)
    g.add_vertex(vertex2)
    g.add_edge(row.split(' '))

In [37]:
#mamba install haversine
info = open('data/locations.txt').read().strip()
import math
from haversine import haversine, Unit
def air_distance(starting_vertex,goal_vertex):
    for row in info.split('\n'):
        vertex,lat,lon=row.split(' ')
        if vertex==starting_vertex:
            start_city=(float(lat),float(lon))
        if vertex==goal_vertex:
            end_city=(float(lat),float(lon))
    return haversine(start_city, end_city)

In [38]:
air_distance('SK','KO')

82.45779267711245

In [43]:
import heapq
def a_star_search(graph, starting_vertex, goal_vertex, alpha=1):
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded:
                    heuristic = air_distance(neighbour, goal_vertex)
                    path_weight = float(current_path_weight) + float(new_weight)
                    a_star_weight = path_weight + alpha * heuristic
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [44]:
a=['SK','KO']
a_star_search(g,'SK', 'KO', alpha=1)

(138.0, ['SK', 'VE', 'ST', 'KO'])